In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from functools import reduce


# Azure Blob Storage SDK
from azure.storage.blob import BlobClient, BlobServiceClient

conn_str = (
    
)

# AzureML SDK
from azureml.core import Workspace, Experiment, Datastore, Dataset, Run

In [117]:
EXPERIMENT_NAME = "my-experiment"

# PROCESS_DATA_CONFIG = json.load(open("process_data_config_v2.json"))
PROCESS_DATA_CONFIG = json.loads("""
        { "demand": {	
        "group": ["Region"],
        "time": "Date",
        "features" : {
            "Weekly_Sales": ["impute_mdian"]}
        }
    }
    """
)

In [120]:
print(PROCESS_DATA_CONFIG)

{'demand': {'group': ['Region'], 'time': 'Date', 'features': {'Weekly_Sales': ['impute_mdian']}}}


In [133]:
with open("process_data_config_v2.json", "r") as f:
    json_file = f.read()
print(json_file)

{ "demand": {	
    "group": ["Region"],
    "time": "Date",
    "features" : {
        "Weekly_Sales": ["impute_mdian"]}
    }
}



In [136]:
str(json_file)

'{ "demand": {\t\n    "group": ["Region"],\n    "time": "Date",\n    "features" : {\n        "Weekly_Sales": ["impute_mdian"]}\n    }\n}\n'

In [135]:
json.loads(json_file)

{'demand': {'group': ['Region'],
  'time': 'Date',
  'features': {'Weekly_Sales': ['impute_mdian']}}}

In [125]:
json.loads("""{ "demand": {	
    "group": ["Region"],
    "time": "Date",
    "features" : {
        "Weekly_Sales": ["impute_mdian"]}
    }
}""")

{'demand': {'group': ['Region'],
  'time': 'Date',
  'features': {'Weekly_Sales': ['impute_mdian']}}}

##  For Script

In [38]:
PROCESS_DATA_CONFIG

{'demand': {'group': ['Region'],
  'time': 'Date',
  'features': {'Weekly_Sales': ['impute_mdian']}}}

In [108]:
ws = Workspace.get(name="Walmart-Sales", 
subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
resource_group = "Walmart-1"
)

experiment = Experiment(ws, name=EXPERIMENT_NAME)

experiment_id = experiment.id

In [15]:
registered_datasets = list(ws.datasets.keys())

In [41]:
PROCESS_DATA_CONFIG

{'demand': {'group': ['Region'],
  'time': 'Date',
  'features': {'Weekly_Sales': ['impute_mdian']}}}

In [93]:
datasets = []

for dataset_name, d in PROCESS_DATA_CONFIG.items():
    if dataset_name in registered_datasets:
        dataset = Dataset.get_by_name(ws, name = dataset_name)
        df = dataset.to_pandas_dataframe()

        GROUP = d['group']
        TIME = d["time"]

         # Formating
        if type(GROUP) == str:
            GROUP = [GROUP]

        # Perform all the necessary steps
        ## TODO: Map the steps to the function to perform transformation

       

        # Append the tuple to the list of datasets
        datasets.append((df, GROUP, TIME))
        
    else:
        print(dataset_name, "not registered")


In [94]:
# datasets.append((df, ["Region"], "Date"))
# datasets.append((df, ["Region"], "Date"))
# datasets.append((df, ["Region"], "Date"))

In [95]:
def merge_dataframe(df1_info, df2_info):
    df1, df1_group, df1_time = df1_info
    df2, df2_group, df2_time = df2_info
    return (pd.merge(df1, df2, how='left', left_on = df1_group + [df1_time], right_on = df2_group + [df2_time]), df1_group, df1_time)

In [111]:
# Merging all datasets

final_df, df_group, df_time = reduce(merge_dataframe, datasets)

In [115]:
########## Upload File to features ###############
FILE_TO_UPLOAD = f"{experiment_id}.csv"

# Storage account name and account key
STORAGE_ACCOUNT_NAME = "walmartsales2005913347"
STORAGE_ACCOUNT_KEY = "uPu7IRa/73JZvkiEBVBAsb8D36g1ZeoRT0YMG7l7ConyJe8aoVKoTwDpDESjZRhs0Mnt6wai7Dwh/IfSAa/B0g=="

conn_string = (
     'DefaultEndpointsProtocol=https;'
    + f'AccountName={STORAGE_ACCOUNT_NAME};'
    + f'AccountKey={STORAGE_ACCOUNT_KEY};'
    + 'EndpointSuffix=core.windows.net'
)

blob_client_upload = BlobClient.from_connection_string(conn_string, 
    container_name="features",
    blob_name= f"{FILE_TO_UPLOAD}"
)

blob_client_upload.upload_blob(
    final_df.to_csv(index=False, header=True).encode()
)

{'etag': '"0x8D9D646E1D48E1D"',
 'last_modified': datetime.datetime(2022, 1, 13, 3, 43, 37, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xfc\x96\xedCj\x0b\xb6\xe3,5\x01\xfdn\xcc\x08K'),
 'client_request_id': 'fd7fd750-7422-11ec-9a25-415645000030',
 'request_id': '8017744a-301e-00a0-712f-086152000000',
 'version': '2019-07-07',
 'date': datetime.datetime(2022, 1, 13, 3, 43, 37, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None,
 'error_code': None}

In [ ]:
def impute_median(df, col):
    

In [99]:
steps_to_func = {
    "impute_median": ""
}